In [1]:
import os
import json
from openai import AzureOpenAI
from azure.identity import ManagedIdentityCredential, get_bearer_token_provider
import os
import requests

token_provider = get_bearer_token_provider(
        ManagedIdentityCredential(), "https://cognitiveservices.azure.com/.default"
        )

client = AzureOpenAI(
        azure_ad_token_provider=token_provider, 
        api_version = "2024-09-01-preview",
        azure_endpoint = "https://aitestepm.openai.azure.com/"
        )

model_name = "gpt-4o" # You need to ensure the version of the model you are using supports the function calling feature
client_id = "fe300394-6c65-467e-9aad-96f2637dbc2d"


credential = ManagedIdentityCredential(client_id=client_id)
token = credential.get_token("d07a4aa0-7148-4e4a-9060-36a8fbec0888/.default")

headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

In [2]:
assistant = client.beta.assistants.create(
  name="Embedding Functions",
  instructions="You are a embeddings bot. Use the provided functions to answer questions.",
  model="gpt-4o", #Replace with model deployment name
  tools=[{
      "type": "function",
    "function": {
      "name": "Call_Embeddings",
      "description": "Uses embeddings",
      "parameters": {
        "type": "object",
        "properties": {
          "Service": {"type": "string", "description": "Service for the query"}
        },
        "required": ["location"]
      }
    }
  }]
)

In [3]:
def Call_Embeddings(Service):
    api_endpoint = "https://logicappembeddingfunction.azurewebsites.net/api/embeddingfunction?Service=" + Service
    response = requests.post(api_endpoint, headers=headers)
    return response._content

In [4]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [5]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Call the embedding function with Service WaAppAgent",
)
thread_messages = client.beta.threads.messages.list(thread.id)





In [6]:
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_s4spMDTm2luhTxwdwaevVFSI",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Call the embedding function with Service WaAppAgent"
          },
          "type": "text"
        }
      ],
      "created_at": 1728897114,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_eELxiR9Qi8pDqsMeM09519Wv"
    }
  ],
  "object": "list",
  "first_id": "msg_s4spMDTm2luhTxwdwaevVFSI",
  "last_id": "msg_s4spMDTm2luhTxwdwaevVFSI",
  "has_more": false
}


In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [8]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

queued


In [9]:
#print(run.model_dump_json(indent=2))
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_s4spMDTm2luhTxwdwaevVFSI",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Call the embedding function with Service WaAppAgent"
          },
          "type": "text"
        }
      ],
      "created_at": 1728897114,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_eELxiR9Qi8pDqsMeM09519Wv"
    }
  ],
  "object": "list",
  "first_id": "msg_s4spMDTm2luhTxwdwaevVFSI",
  "last_id": "msg_s4spMDTm2luhTxwdwaevVFSI",
  "has_more": false
}


In [10]:
for tool in run.required_action.submit_tool_outputs.tool_calls:
    print(tool)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [11]:
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  # get data from the weather function
  if tool.function.name == "Call_Embeddings":
    arguments = json.loads(tool.function.arguments)
    result = Call_Embeddings(**arguments)
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": result
    })
 
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  print("run status: ", run.status)
  messages = client.beta.threads.messages.list(thread_id=thread.id)
  print(messages.to_json(indent=2))

else:
  print("run status: ", run.status)
  print (run.last_error.message)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [12]:
tool_outputs

[]

In [ ]:
Call_Embeddings("WaAppAgent")